# inference

In [1]:
import sys
print(sys.version)
print("현재 가상 환경 경로:", sys.prefix)
print(sys.executable)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
현재 가상 환경 경로: c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\myenv38
c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\myenv38\Scripts\python.exe


## precession

### package setting

In [2]:
import torch
import albumentations as albu
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

import segmentation_models_pytorch as smp

import numpy as np
import pandas as pd 
from PIL import Image
import shutil
import os
import random
from tqdm.auto import tqdm
import cv2
import matplotlib.pyplot as plt

c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\myenv38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\myenv38\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 42
seed_everything(seed)

### definition

#### 모델

In [4]:
def rebuild_dir(target_path):
    if os.path.exists(target_path):
        shutil.rmtree(target_path)
        os.makedirs(target_path)
    else:
        os.makedirs(target_path)
def build_dir(target_path):
    if not os.path.exists(target_path):
        os.makedirs(target_path)

In [5]:
def cropping_image(df, save_path):
    print("전체 사진 수: ", len(df))
    size = 512
    for idx in (df.index):
        base_names = df.loc[idx,'base_names']    
        img_path = df.loc[idx,'file_dir']

        img_rgb=cv2.imread(img_path, cv2.IMREAD_COLOR)  # 원본이미지
        max_height,max_width = img_rgb.shape[0],img_rgb.shape[1]
        
        num=0
        
        label_dir = df.loc[idx,'autolabeling_dir']
        result_dir = df.loc[idx,'img_dir']

        rebuild_dir(label_dir)    
        build_dir(result_dir)      

        for height in tqdm(range(0, img_rgb.shape[0], size)):
            for width in range(0, img_rgb.shape[1], size):

                img_rgb_crop = img_rgb[height:height+size, width:width+size, :]  # 512*512 절삭(높이, 너비)
                if height+size > max_height or width+size > max_width:
                    pass
                else:
                    name = str(base_names)+'_Cropped_'+'%05d.png' % num
                    tile_dir = os.path.join(save_path, name)
                    cv2.imwrite(tile_dir,img_rgb_crop)
                    num += 1
                    
        print(f"파일명: {df.loc[idx,'file_name']}\nSaved cropped image: {num}")


In [6]:
class InferenceDataset(BaseDataset):
    CLASSES = ['x' for x in range(255)]
    CLASSES.append('cell')

    def __init__(
        self,
        images_dir,
        images_files,
        classes=None,
        preprocessing=None,
    ):
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in (images_files)]  # image_path
        self.preprocessing = preprocessing
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.tile_size = 512

        # 전체 이미지에서 모든 타일 정보를 미리 생성
        self.tiles_info = []
        for img_idx, image_path in enumerate(self.images_fps):
            image = cv2.imread(image_path)
            height, width, _ = image.shape
            
            for h in range(0, height, self.tile_size):
                for w in range(0, width, self.tile_size):
                    self.tiles_info.append({
                        'img_idx': img_idx, # 원본 이미지 인덱스
                        'h_start': h,
                        'w_start': w,
                    })

    def __getitem__(self, i):
        # i번째 타일 정보 가져오기
        tile_info = self.tiles_info[i]
        img_idx = tile_info['img_idx']
        h_start = tile_info['h_start']
        w_start = tile_info['w_start']
        
        # 원본 이미지 로드
        image_path = self.images_fps[img_idx]
        image = cv2.imread(image_path)
        # 흑백일 때(2D array) BGR로 바꿔 주기
        if image.ndim == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        else:
            # 컬러 파일이라면 기본 BGR 로드 → RGB 로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        # 타일 크롭
        tile = image[h_start:h_start+self.tile_size, w_start:w_start+self.tile_size]
        
        # 전처리 적용
        if self.preprocessing:
            sample = self.preprocessing(image=tile)
            tile = sample['image']

        # 원본 이미지의 이름과 크기 정보 반환
        original_image_name = os.path.splitext(os.path.basename(image_path))[0]    
        original_height, original_width, _ = image.shape
        
        return tile, h_start, w_start, original_height, original_width, original_image_name

    def __len__(self):
        return len(self.tiles_info)

In [7]:
def to_tensor(x, **kwargs):
    if x.ndim == 2:  # 채널이 없는 경우    
        x = np.expand_dims(x, axis=-1)  # (H, W) → (H, W, 1)
    if x.shape[2] == 1:  # 1채널일 경우 → 3채널로 반복
        x = np.repeat(x, 3, axis=2)
    x = x.transpose(2, 0, 1).astype('float32')  # (C, H, W)
    return torch.from_numpy(x)

def get_preprocessing(preprocessing_fn):

    _transform = [
        albu.PadIfNeeded(512, 512, always_apply=True, border_mode=cv2.BORDER_CONSTANT, value=0),
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

In [8]:
def inference_collate_fn(batch):
    """
    batch: [(tile, h, w, H, W, name),  … ]  형태의 리스트
    -> ([tiles…], [h_starts…], [w_starts…], [heights…], [widths…], [names…])
    """
    tiles, h_starts, w_starts, heights, widths, names = zip(*batch)
    tiles_batch = torch.stack(tiles)
    # tiles: tuple of tensors, h_starts 등은 tuple of int, names는 tuple of str
    return (tiles_batch), list(h_starts), list(w_starts), list(heights), list(widths), list(names)

#### mask

In [9]:
def has_less_white_pixels(image_path, threshold = 1):
    """
    이미지 내에 지정된 threshold 값 미만의 흰색 픽셀이 있는지 확인합니다.
    :param image_path: 이미지 파일의 경로
    :param threshold: 흰색 픽셀의 최대 수
    :return: 지정된 threshold 값을 미만의 흰색 픽셀이 있으면 True, 그렇지 않으면 False
    """
    white_pixel_count = 0
    try:
        with Image.open(image_path) as img:
            # 이미지 모드 확인: 'L' 모드는 흑백, 'RGB'는 컬러
            if img.mode != 'L':
                # 흑백이 아니면 흑백으로 변환하여 픽셀 값 비교를 용이하게 합니다.
                img = img.convert('L')
            
            pixels = img.getdata()
            
            # 각 픽셀 값이 255(흰색)인지 직접 확인
            for pixel in pixels:
                if pixel == 255:
                    white_pixel_count += 1
                if white_pixel_count >= threshold:
                    return False
        return True
    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {e}")
        return False # 오류 발생 시 False 반환

In [10]:
def blurrd_img(tiff_files):
    ## 원본 이미지
    img = cv2.imread(tiff_files, cv2.IMREAD_COLOR)

    ## 블러 처리
    blur = cv2.medianBlur(img, 9)

    hsv_img = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    ## hsv 채널분리
    h, s, v = cv2.split(hsv_img)

    _, thr_s = cv2.threshold(s, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    __cached__, thr_v = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    thr_v -= 255

    # Contours 찾기
    contours_s, _ = cv2.findContours(thr_s, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_v, _ = cv2.findContours(thr_v, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    contour_img_s = np.zeros_like(thr_s)
    cn_filled_s = cv2.drawContours(contour_img_s, contours_s, -1, (255,255,255), -1)

    contour_img_v = np.zeros_like(thr_v)
    cn_filled_v = cv2.drawContours(contour_img_v, contours_v, -1, (255,255,255), -1)

    ## hsv 채널 중 s,v 채널 영역 합침
    mask_area = cv2.bitwise_and(cn_filled_s, cn_filled_v)

    # Contours 찾기
    contours, _ = cv2.findContours(mask_area, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Contours를 그릴 이미지 생성
    contour_img = np.zeros_like(mask_area)

    # Contours 그리기
    cn_filled = cv2.drawContours(contour_img, contours, -1, (255,255,255), -1)

    result_img = img.copy()
    result_img[cn_filled==0] = [255, 255, 255]  # 흰색으로 채우기

    return result_img, mask_area

In [11]:
def count_pixels_in_mask(mask):
    """마스크 안의 픽셀 수를 반환합니다."""
    return np.sum(mask)

def draw_contours_on_image(image, mask, color, full_mask=None):
    """
    Mask의 테두리를 이미지에 그립니다.
    (수정) full_mask가 제공되면, full_mask 영역 안에 포함된 contour만 그립니다.
    """
    
    # 1. 예측 마스크(U-Net)에서 모든 contour 찾기
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    image_with_contours = image.copy()
    
    filtered_contours = []
    
    if full_mask is not None:
        # 2. 필터링 로직: full_mask 영역 안에 있는 contour만 선택
        for cnt in contours:
            try:
                # 3. 예측 contour의 중심점 계산
                M = cv2.moments(cnt)
                if M['m00'] == 0: continue # 면적이 0인 contour 방지
                
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])

                # 4. 중심점이 full_mask(blurrd_img 결과)의 흰색 영역(255)에 있는지 확인
                # (배열 인덱싱은 y, x 순서)
                if full_mask[cy, cx] != 0:
                    filtered_contours.append(cnt)
            except Exception as e:
                # 중심점 계산이 불가능한 아주 작은 contour 등 예외 처리
                pass 
    else:
        # full_mask가 없으면 모든 contour를 그림
        filtered_contours = contours
        
    # 5. 필터링된 contour만 원본 이미지에 그리기
    cv2.drawContours(image_with_contours, filtered_contours, -1, color, 10)
    return image_with_contours


#### histogram

In [12]:
def analyze_segmentation_mask(mask_path, bin_size=1000):
    """
    저장된 전체 마스크 이미지를 분석하여 Contour 개수와 크기 분포를 반환합니다.

    :param mask_path: 분석할 마스크 이미지 파일 경로 (예: '..._mask.png')
    :param bin_size: 히스토그램의 구간 크기 (기본값: 1000 픽셀)
    :return: (전체 Contour 개수, 히스토그램 카운트, 히스토그램 구간) 튜플
    """
    
    # 1. 마스크 이미지를 그레이스케일로 로드
    mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask_image is None:
        print(f"오류: 이미지를 로드할 수 없습니다. 경로: {mask_path}")
        return 0, None, None

    # 2. Contour 찾기
    # cv2.RETR_EXTERNAL: 가장 바깥쪽의 Contour만 찾습니다.
    # cv2.CHAIN_APPROX_SIMPLE: Contour의 꼭짓점만 저장하여 메모리를 절약합니다.
    contours, _ = cv2.findContours(
        mask_image, 
        cv2.RETR_EXTERNAL, 
        cv2.CHAIN_APPROX_SIMPLE
    )
    
    total_contours = len(contours)
    print(f"--- 마스크 분석 결과 ('{os.path.basename(mask_path)}') ---")
    print(f"✅ 전체 Contour 개수: {total_contours} 개")
    
    if total_contours == 0:
        print("분석할 Contour가 없습니다.")
        return 0, None, None

    # 3. 각 Contour의 픽셀 면적(Area) 계산
    contour_areas = [cv2.contourArea(cnt) for cnt in contours]
    
    # 4. 1000 단위로 찢어서 히스토그램화
    max_area = max(contour_areas)
    
    # 히스토그램 구간(bins) 설정
    # 예: max_area가 3500이면, bins = [0, 1000, 2000, 3000, 4000]
    bins = np.arange(0, max_area + bin_size, bin_size)
    
    # NumPy를 사용하여 히스토그램 계산
    hist_counts, bin_edges = np.histogram(contour_areas, bins=bins)
    
    # # 5. 결과 출력
    # print("\n--- Contour 크기 분포 (히스토그램) ---")
    # for i in range((total_contours)):
    #     print(f"{i}번째 contour 크기: {contour_areas[i]} 픽셀")
        
    return total_contours, hist_counts, bin_edges


In [13]:
def plot_histogram_skip_empty(hist_counts, bin_edges, save_path ,name):
    """
    계산된 히스토그램 데이터에서 카운트가 0인 구간을 '건너뛰고' (제외하고)
    막대그래프로 시각화하고 저장합니다.
    """
    
    if hist_counts is None or np.sum(hist_counts) == 0:
        print("시각화할 히스토그램 데이터가 없습니다.")
        return

    # --- 수정된 부분: 10이하인 구간 필터링 ---
    
    # 1. 카운트가 0보다 큰(> 0) 구간의 인덱스를 찾습니다.
    non_zero_indices = np.where(hist_counts > 10)[0]
    
    if len(non_zero_indices) == 0:
        print("시각화할 데이터가 없습니다 (모든 구간의 카운트가 0입니다).")
        return

    # 2. 0이 아닌 카운트만 필터링합니다.
    filtered_counts = hist_counts[non_zero_indices]
    
    # 3. 필터링된 구간에 해당하는 Bin 레이블을 생성합니다.
    # 예: "0-1000", "2000-3000" (1000-2000이 비어있다면 건너뜀)
    bin_labels = []
    for i in non_zero_indices:
        label = f"{int(bin_edges[i])}-{int(bin_edges[i+1])}"
        bin_labels.append(label)
    # --- 여기까지 수정 ---

    # 4. 필터링된 데이터로 막대그래프를 생성합니다.
    # 그래프의 너비를 데이터 개수에 따라 동적으로 조절
    plt.figure(figsize=(max(10, len(bin_labels) * 0.5), 6))
    plt.bar(bin_labels, filtered_counts, color='skyblue', edgecolor='black')
    
    plt.title('Histogram of Contour Areas (10 more section)')
    plt.xlabel('Pixel Area Range (px)')
    plt.ylabel('Number of Contours (Count)')
    # plt.xticks(rotation=45, ha='right') # 레이블이 겹치지 않게 회전
    plt.tight_layout() # 레이아웃 최적화
    
    # 5. 그래프를 이미지 파일로 저장
    plt.savefig(os.path.join(save_path,f'{name}_histogram.png'))
    plt.show()
    print(f"\n✅ (카운트가 10미만인 구간 제외) 히스토그램 그래프가 '{save_path}'에 저장되었습니다.")



## main test

### path

In [14]:
current_path = os.getcwd()  # root
dataset_name = 'Full_Image'
ckpt_path = os.path.join(current_path, 'ckpt')

# 테스트용
image_path = os.path.abspath(current_path + "/../../../test")
images = os.listdir(image_path)[:2]

In [17]:
# 테스트용
image_path = os.path.abspath(current_path + "/../../../school/CAPSTONE/main/model/Full_Image/original_data/TG/TIFF")
images = os.listdir(image_path)
print(image_path)

c:\working\school\CAPSTONE\main\model\Full_Image\original_data\TG\TIFF


In [18]:
image_pathes = [os.path.join(image_path, x) for x in images]  # 시스템에서는 입력으로 받음
image_path = image_pathes[0].split('\\')[:-1]
image_dir = '\\'.join(image_path)
image_file = [image.split("\\")[-1] for image in image_pathes]

print(ckpt_path)
print(image_pathes)
print(image_dir)
print(image_file)

c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\System\ckpt
['c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\1.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\10.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\11.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\12.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\13.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\14.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\2.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\3.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\4.tif', 'c:\\working\\school\\CAPSTONE\\main\\model\\Full_Image\\original_data\\TG\\TIFF\\5.tif'

In [22]:
sample_path = os.path.abspath(current_path+"/../results")

sample_crop = os.path.join(sample_path,'preprocessing','Crop')  # 자른 이미지
sample_label = os.path.join(sample_path,'preprocessing','Label')  # 자른 라벨

sample_prediction = os.path.join(sample_path,'prediction')  # 결과 이미지들
sample_segmentation = os.path.join(sample_path,'segmentation')  # 결과 마스크들
print(sample_path)

c:\working\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results


In [23]:
rebuild_dir(sample_path)

In [22]:
pd.set_option('display.max_colwidth', 5500)

base_name = ['.'.join(x.split('.')[:-1]) for x in image_file]
df = pd.DataFrame({
    'base_names': base_name,
    'file_name': image_file,
    'file_dir': [os.path.join(image_dir, x) for x in image_file],
    'autolabeling_dir': [os.path.join(sample_label, f'{x}_crop') for x in base_name],  # 라벨링 결과
    'img_dir': [os.path.join(sample_prediction, f'{x}_result') for x in base_name]})  # 결과 데이터
df

,base_names,file_name,file_dir,autolabeling_dir,img_dir
0,1,1.tif,c:\Users\yy\Desktop\test\1.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\1_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\1_result
1,10,10.tif,c:\Users\yy\Desktop\test\10.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\10_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\10_result
2,11,11.tif,c:\Users\yy\Desktop\test\11.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\11_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\11_result
3,12,12.tif,c:\Users\yy\Desktop\test\12.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\12_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\12_result
4,13,13.tif,c:\Users\yy\Desktop\test\13.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\13_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\13_result
5,14,14.tif,c:\Users\yy\Desktop\test\14.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\14_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\14_result
6,2,2.tif,c:\Users\yy\Desktop\test\2.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\2_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\2_result
7,3,3.tif,c:\Users\yy\Desktop\test\3.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\3_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\3_result
8,4,4.tif,c:\Users\yy\Desktop\test\4.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\4_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\4_result
9,5,5.tif,c:\Users\yy\Desktop\test\5.tif,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\preprocessing\Label\5_crop,c:\Users\yy\Desktop\hbnu-swuniv-capstone-project-come-capstone25-CAPSTONE-InnoMedics\003Code\results\prediction\5_result


### tile, model setting

In [23]:
rebuild_dir(sample_crop)
cropping_image(df, sample_crop)

전체 사진 수:  14


100%|██████████| 43/43 [00:16<00:00,  2.65it/s]


파일명: 1.tif
Saved cropped image: 1680


100%|██████████| 36/36 [00:15<00:00,  2.29it/s]


파일명: 10.tif
Saved cropped image: 1296


...

파일명: 7.tif
Saved cropped image: 272


100%|██████████| 19/19 [00:03<00:00,  6.11it/s]


파일명: 8.tif
Saved cropped image: 342


100%|██████████| 35/35 [00:18<00:00,  1.92it/s]

파일명: 9.tif
Saved cropped image: 1462


In [24]:
ENCODER = 'efficientnet-b2'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['cell']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
# create segmentation model with pretrained encoder
model = smp.UnetPlusPlus(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=len(CLASSES),
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
print(DEVICE)

cpu


In [25]:
# Create test dataset
test_dataset = InferenceDataset(
    images_dir=image_dir,
    images_files=image_file,
    classes=CLASSES,
    # augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
)

In [26]:
test_dataloader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, # 한 번에 하나의 원본 이미지(의 타일들)를 처리
    shuffle=False, 
    num_workers=0,  # num_workers/는 시스템의 CPU 코어 수를 활용할 수 있지만, 여기서는 0으로 설정하여 단일 프로세스에서 실행
    # num_workers=os.cpu_count(), # 시스템의 CPU 코어 수 활용
    collate_fn=inference_collate_fn
)

In [27]:
best_model = torch.load(os.path.join(ckpt_path,'{}_model2.pth'.format(dataset_name)), map_location=DEVICE)

C:\Users\yy\AppData\Local\Temp\ipykernel_21912\573539083.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(os.path.join(ckpt_path,'{}_model2.pth'.f

### blur

In [ ]:
for i in range(len(df)):
    print(df.loc[i,'file_name'])
    result_path = df.loc[i,'img_dir']
    name = df.loc[i,'base_names']

    blurrd_path = os.path.join(result_path, f'{name}_blurrd.png')


### inference

In [ ]:
@torch.no_grad() # 추론 시에는 그래디언트 계산을 비활성화하여 메모리 사용량을 줄이고 속도를 높입니다.
def inference_on_folder(model, dataloader, device, inference_dir):
    model.eval().to(device)
    progress_bar = tqdm(dataloader, desc="Inference Progress")
    
    full_masks = {}
    num = 0
    
    for i, data in enumerate(progress_bar):
        tiles_batch, h_starts, w_starts, heights, widths, image_names = data
        #print(f"\n[{i+1}/{len(dataloader)}] 배치 추론 중...")        
        tiles_batch = (tiles_batch).to(device)  
        
        # 모델 예측
        prediction = model(tiles_batch) # 출력: (batch_size, num_classes, H, W)
        predictions = prediction.cpu().numpy() 
        
        for j in range(len(image_names)):
            img_name = image_names[j]
            h_start = h_starts[j]
            w_start = w_starts[j]
            height = heights[j]
            width = widths[j]
            predicted_mask_tile = predictions[j]
            # 활성화 함수 적용 후 CPU로 이동하고 NumPy 배열로 변환
            # sigmoid 활성화 함수를 사용하면 [0, 1] 범위의 확률 맵이 출력됩니다.
            # 'cell' 클래스 하나만 예측하므로 prediction.squeeze()를 사용합니다.
            # threshold를 적용하여 이진 마스크로 변환 (0.5 이상이면 1, 아니면 0)
            
            # 단일 클래스 이진 세그멘테이션의 경우 (1, H, W) -> (H, W)
            if len(CLASSES) == 1:
                predicted_mask_tile = predicted_mask_tile.squeeze(0) # (H, W)
                predicted_mask_tile = (predicted_mask_tile > 0.5).astype(np.uint8) * 255 # 이진 마스크로 변환 (0 또는 255)
            else:
                # 다중 클래스 세그멘테이션의 경우 argmax 사용
                predicted_mask_tile = np.argmax(predicted_mask_tile, axis=0).astype(np.uint8) # (H, W)
                # 클래스 ID를 실제 픽셀 값으로 매핑해야 할 수 있습니다. (예: 0, 1, 2... 에 따라 다른 색상)
                # 여기서는 예시로 255로 스케일링하여 저장합니다.
                predicted_mask_tile = predicted_mask_tile * (255 // (len(CLASSES) -1)) if len(CLASSES) > 1 else predicted_mask_tile * 255
            tile_path = os.path.join(inference_dir,f"{img_name}_crop", f"{num:05d}.png")
            cv2.imwrite(tile_path, predicted_mask_tile)     
            
            if has_less_white_pixels(tile_path):
                os.remove(tile_path)
                continue
            num += 1

            # --- 전체 마스크 재구성 로직 ---
            # 현재 이미지에 대한 전체 마스크가 딕셔너리에 없으면 초기화
            if img_name not in full_masks:
                full_masks[img_name] = np.zeros((height, width), dtype=np.uint8)
            
            h_end = min(h_start + 512, height)
            w_end = min(w_start + 512, width)
            
            # 재구성할 마스크의 실제 높이와 너비
            actual_reconstruct_height = h_end - h_start
            actual_reconstruct_width = w_end - w_start
            
            # 예측된 타일 마스크에서 원본 영역에 해당하는 부분만 추출
            reconstructed_tile = predicted_mask_tile[:actual_reconstruct_height, :actual_reconstruct_width]
            
            # 전체 마스크에 재구성
            full_masks[img_name][h_start:h_end, w_start:w_end] = reconstructed_tile
        # 현재 배치의 추론 진행 상황 출력
        #print(f"[{i+1}/{len(dataloader)}] 배치 추론 완료.")

    # 모든 배치가 처리된 후, 최종 마스크들을 파일로 저장
    print("\n모든 이미지 타일 추론이 완료되었습니다. 최종 결과를 저장합니다. 💾") 
    return full_masks
    
# 추론 실행
full_masks = inference_on_folder(best_model, test_dataloader, DEVICE, sample_label)


Inference Progress:   0%|          | 0/382 [00:00<?, ?it/s]

Inference Progress:   5%|▌         | 21/382 [12:50<3:40:53, 36.71s/it]


KeyboardInterrupt: 

In [ ]:
idx = 0
for img_name,full_mask in (full_masks.items()):
    image = cv2.imread(os.path.join(df.loc[idx, 'file_dir']))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    output_dir = os.path.join(df.loc[idx, 'img_dir'], f"{img_name}_contours.png")
    result_dir = os.path.join(sample_segmentation, f"{img_name}_segmentation.png")


    blurrd_path = os.path.join(df.loc[idx, 'img_dir'], f'{img_name}_blurrd.png')
    blurrd_image = cv2.imread(blurrd_path, cv2.IMREAD_COLOR)
    image_with_prediction_contours = draw_contours_on_image(       image, full_mask.astype(np.uint8), (0, 255, 0))  # 초록색 prediction 테두리
        
    idx += 1
    
    cv2.imwrite(output_dir, image_with_prediction_contours)
    cv2.imwrite(result_dir, full_mask)
    print(f"'{img_name}'의 예측 마스크를 '{output_dir}'에 저장했습니다. ✅")